In [ ]:

# TIPS: PARAMS + HELPERS 


library(dplyr)
library(readr)
library(tuneR)

#  RUTAS
OUT_ROOT <- "../results/asr_KNN/SYNTH_TIPS"
MANIFEST_CSV <- "../results/asr_KNN/NN_AUDIO/ORIG/TIPS/neighbors_manifest_ORIG_TIPS.csv"

#  Parámetros espectrograma 
TLIM <- c(0, 5)     # segundos
FLIM <- c(0, 10)    # kHz 
WL   <- 512
OVLP <- 75

# Regrilla (para export estable)
X_LENGTH <- 140
Y_LENGTH <- 94

#  Top-K máximo (por tip)
K_USE <- 16

#  Vecinos solo de la MISMA especie del TIP 
SAME_SPECIES_ONLY <- TRUE

#  Columnas del manifest (neighbors_manifest_ORIG_TIPS.csv) 
COL_TIP    <- "tip"       # nombre especie del tip (con espacios)
COL_RANK   <- "rank"      # ranking del vecino
COL_WAVIDX <- "wav_index" # índice (trazabilidad)
COL_DIST   <- "distance"  # distancia en embedding
COL_SRCWAV <- "src_wav"   # ruta original del wav
COL_DSTWAV <- "dst_wav"   # ruta del wav copiado (este se usa para leer audio)

#  Selección de TIPS a procesar 
MODE <- "all"          # "one" | "some" | "all"
TIP_ONE <- "Tinamus major"  # 
TIP_SOME <- c("Tinamus major", "Crypturellus tataupa")

#  Pesos (ponderación por distancia)
# Pesos: w_i = exp(-(d_i/tau)^2)
kernel_w <- function(d, tau) exp(- (d / tau)^2 )


# HELPERS AUDIO / STFT

read_wav_num <- function(path) {
  w <- tuneR::readWave(path)
  y <- as.numeric(w@left) / (2^(w@bit - 1))
  list(y = y, fs = w@samp.rate)
}

fade_in_out <- function(y, fs, fade_ms = 20) {
  n <- as.integer(round(fs * (fade_ms/1000)))
  if (n > 0 && 2*n < length(y)) {
    y[1:n] <- y[1:n] * seq(0, 1, length.out = n)
    y[(length(y)-n+1):length(y)] <- y[(length(y)-n+1):length(y)] * seq(1, 0, length.out = n)
  }
  y
}

stft_power <- function(y, n_fft, hop, win) {
  n <- length(y)
  n_frames <- as.integer(floor((n - n_fft) / hop) + 1)
  P <- matrix(0, nrow = n_fft/2 + 1, ncol = n_frames)
  for (j in 1:n_frames) {
    i0 <- (j - 1) * hop + 1
    frame <- y[i0:(i0 + n_fft - 1)] * win
    F <- fft(frame)[1:(n_fft/2 + 1)]
    P[, j] <- (Mod(F)^2)
  }
  P
}

# STFT params (cálculo en 0–10 kHz)
N_FFT <- WL
HOP   <- as.integer(round(N_FFT * (1 - OVLP/100)))
WIN   <- 0.5 - 0.5*cos(2*pi*(0:(N_FFT-1))/(N_FFT-1))

# Regrillar a (Y_LENGTH × X_LENGTH) en 0–10 kHz
regrid_power_to_mesh <- function(P_avg, fs, tlim, flim, x_len, y_len) {
  freqs_hz <- (0:(N_FFT/2)) * fs / N_FFT
  times_s  <- (0:(ncol(P_avg)-1)) * HOP / fs

  # recorte cálculo hasta flim[2]
  fmax_hz <- flim[2] * 1000
  kmax <- max(which(freqs_hz <= fmax_hz))
  P_avg <- P_avg[1:kmax, , drop = FALSE]
  freqs_hz <- freqs_hz[1:kmax]

  t_mesh <- seq(tlim[1], tlim[2], length.out = x_len)
  f_mesh_hz <- seq(flim[1]*1000, flim[2]*1000, length.out = y_len)

  P_t <- matrix(0, nrow = nrow(P_avg), ncol = x_len)
  for (r in 1:nrow(P_avg)) {
    P_t[r, ] <- approx(times_s, P_avg[r, ], xout = t_mesh, rule = 2)$y
  }

  P_grid <- matrix(0, nrow = y_len, ncol = x_len)
  for (j in 1:x_len) {
    P_grid[, j] <- approx(freqs_hz, P_t[, j], xout = f_mesh_hz, rule = 2)$y
  }

  list(P_grid = P_grid, t_mesh = t_mesh, f_mesh_hz = f_mesh_hz)
}

In [ ]:
# LIMPIA — TIPS


# Cargar manifest
manifest <- readr::read_csv(MANIFEST_CSV, show_col_types = FALSE)

# Subset de tips
if (MODE == "one") {
  manifest <- manifest %>% dplyr::filter(.data[[COL_TIP]] == TIP_ONE)
} else if (MODE == "some") {
  manifest <- manifest %>% dplyr::filter(.data[[COL_TIP]] %in% TIP_SOME)
}

# Filtro MISMA ESPECIE
if (SAME_SPECIES_ONLY) {
  manifest <- manifest %>%
    dplyr::mutate(
      tip_species     = gsub("_", " ", .data[[COL_TIP]]),
      neigh_species   = gsub("_", " ", basename(dirname(.data[[COL_SRCWAV]]))),
      same_species_ok = (neigh_species == tip_species)
    ) %>%
    dplyr::filter(same_species_ok)
}

# Top-K por tip (rank si está; si no, distance)
if (COL_RANK %in% names(manifest)) {
  manifest_k <- manifest %>%
    dplyr::arrange(.data[[COL_TIP]], .data[[COL_RANK]]) %>%
    dplyr::group_by(.data[[COL_TIP]]) %>%
    dplyr::slice_head(n = K_USE) %>%
    dplyr::ungroup()
} else {
  manifest_k <- manifest %>%
    dplyr::arrange(.data[[COL_TIP]], .data[[COL_DIST]]) %>%
    dplyr::group_by(.data[[COL_TIP]]) %>%
    dplyr::slice_head(n = K_USE) %>%
    dplyr::ungroup()
}

# Diagnóstico: ¿cuántos vecinos quedaron por tip?
count_by_tip <- manifest_k %>%
  dplyr::count(.data[[COL_TIP]], name = "n_kept") %>%
  dplyr::arrange(n_kept)
print(count_by_tip)

# Diagnóstico distancias + tau sugerido por tip (p90)
if (!(COL_DIST %in% names(manifest_k))) stop("No encuentro columna distance en el manifest")

dist_summary <- manifest_k %>%
  dplyr::group_by(.data[[COL_TIP]]) %>%
  dplyr::summarise(
    n = dplyr::n(),
    d_min = min(.data[[COL_DIST]], na.rm = TRUE),
    d_med = median(.data[[COL_DIST]], na.rm = TRUE),
    d_p90 = quantile(.data[[COL_DIST]], 0.90, na.rm = TRUE, names = FALSE),
    d_p95 = quantile(.data[[COL_DIST]], 0.95, na.rm = TRUE, names = FALSE),
    .groups = "drop"
  ) %>%
  dplyr::arrange(d_med)

print(dist_summary)

TAU_SUGGEST_PER_TIP <- dist_summary %>%
  dplyr::select(tip = all_of(COL_TIP), tau = d_p90) %>%
  dplyr::mutate(tip = gsub("_", " ", tip))

# Decisión: TAU POR TIP 
TAU_MODE <- "per_tip"
TAU_LOOKUP <- setNames(TAU_SUGGEST_PER_TIP$tau, TAU_SUGGEST_PER_TIP$tip)
cat("TAU_MODE = ", TAU_MODE, " (usando p90 por tip)\n", sep = "")


In [ ]:
# TIPS: WEIGHTED EIGENSOUND (ALIGNED) + EXPORT CSVs

library(SoundShape)
library(geomorph)

DBLEVEL_EIG <- 25
LOG_SCALE   <- FALSE
CSV_SUFFIX  <- "_wtd"

n_eff <- function(w) (sum(w)^2) / sum(w^2)

wtd_quantile <- function(x, w, probs = c(0.025, 0.975)) {
  o <- order(x)
  x <- x[o]; w <- w[o]
  cw <- cumsum(w) / sum(w)
  sapply(probs, function(p) x[which(cw >= p)[1]])
}

w_mshape <- function(A, w) {
  apply(A, c(1,2), function(v) sum(v * w) / sum(w))
}

#  PCA ponderado (con consistencia de signo en PC1)
wpca <- function(X, w, n_pc = 10) {
  w <- as.numeric(w)
  w_norm <- w / sum(w)

  mu <- as.numeric(colSums(X * w_norm))
  Xc <- sweep(X, 2, mu, "-")

  Xw <- Xc * sqrt(w_norm)

  r <- min(n_pc, nrow(X) - 1, ncol(X))
  sv <- svd(Xw, nu = 0, nv = r)

  rotation <- sv$v
  values   <- sv$d^2
  totalvar <- sum(Xw^2)
  scores   <- Xc %*% rotation

  if (ncol(scores) >= 1) {
    totE <- rowSums(X)

    mx <- sum(w_norm * scores[, 1])
    my <- sum(w_norm * totE)

    cov_xy <- sum(w_norm * (scores[, 1] - mx) * (totE - my))
    var_x  <- sum(w_norm * (scores[, 1] - mx)^2)
    var_y  <- sum(w_norm * (totE - my)^2)

    cor_w <- cov_xy / sqrt(var_x * var_y)

    if (is.finite(cor_w) && cor_w < 0) {
      scores[, 1]   <- -scores[, 1]
      rotation[, 1] <- -rotation[, 1]
    }
  }

  list(mu = mu, rotation = rotation, values = values, totalvar = totalvar, scores = scores)
}

tips <- sort(unique(manifest_k[[COL_TIP]]))

for (tip in tips) {

  tip_id  <- gsub(" ", "_", tip)
  tip_dir <- file.path(OUT_ROOT, paste0("tip_", tip_id))

  ali.full <- file.path(tip_dir, "wav.at", "Aligned")
  store.at <- file.path(tip_dir, "store.at")
  dir.create(store.at, recursive = TRUE, showWarnings = FALSE)

  ali.tmp <- file.path(tip_dir, "_tmp_aligned_topK")
  unlink(ali.tmp, recursive = TRUE)
  dir.create(ali.tmp, recursive = TRUE, showWarnings = FALSE)

  # WAVs alineados del tip
  if (!dir.exists(ali.full)) {
    stop(paste0("No existe carpeta Aligned para tip: ", tip, " -> ", ali.full))
  }

  wavs <- sort(list.files(ali.full, pattern="\\.wav$", full.names=TRUE))

  # Mantiene solo los wavs cuyo basename contiene el tip_id (Genus_species)
  wavs <- wavs[grepl(tip_id, basename(wavs), fixed = TRUE)]

  if (length(wavs) == 0) {
    stop(paste0("Aligned existe pero no encontré wavs de la especie del tip dentro de: ", ali.full,
                " (busqué tip_id=", tip_id, " en el filename)."))
  }

  # top-K (hasta donde llegue)
  wavs_k <- wavs[1:min(K_USE, length(wavs))]
  file.copy(wavs_k, ali.tmp, overwrite = TRUE)

  # distancias di desde filename (idéntico)
  wav_names <- basename(wavs_k)
  d <- as.numeric(sub(".*_d([0-9.]+)_.*", "\\1", wav_names))

  # TAU por tip
  tau_use <- TAU_LOOKUP[[tip]]
  if (is.null(tau_use) || is.na(tau_use)) stop(paste0("No hay TAU para tip: ", tip))

  w <- kernel_w(d, tau_use)
  neff <- n_eff(w)

  cat("\n=== TIP", tip, "| K =", length(wavs_k),
      "| tau =", signif(tau_use, 6),
      "| n_eff =", signif(neff, 4), "===\n")
  flush.console()

  res <- try({

    EIG3D <- SoundShape::eigensound(
      analysis.type = "threeDshape",
      wav.at        = ali.tmp,
      store.at      = store.at,
      flim          = FLIM,
      tlim          = TLIM,
      wl            = WL,
      ovlp          = OVLP,
      dBlevel       = DBLEVEL_EIG,
      x.length      = X_LENGTH,
      y.length      = Y_LENGTH,
      log.scale     = LOG_SCALE,
      plot.exp      = FALSE
    )

    p <- dim(EIG3D)[1]
    k <- dim(EIG3D)[2]

    # CONSENSO ponderado
    ref_w <- w_mshape(EIG3D, w)
    Mmean <- matrix(ref_w[,3], nrow = Y_LENGTH, ncol = X_LENGTH, byrow = TRUE)
    write.csv(Mmean, file.path(store.at, paste0("Mmean", CSV_SUFFIX, ".csv")), row.names = FALSE)

    # PCA ponderado
    Xmat <- geomorph::two.d.array(EIG3D)
    PCAw <- wpca(Xmat, w)

    pve <- PCAw$values / PCAw$totalvar

    pve12 <- data.frame(
      PC = c("PC1","PC2"),
      pve = c(pve[1], pve[2]),
      cum = c(pve[1], pve[1] + pve[2])
    )
    write.csv(pve12, file.path(store.at, paste0("PVE_PC1_PC2", CSV_SUFFIX, ".csv")), row.names = FALSE)

    # cuantiles ponderados de scores (PC1/PC2)
    s1 <- PCAw$scores[,1]
    s2 <- PCAw$scores[,2]

    q1 <- wtd_quantile(s1, w, probs = c(0.025, 0.975))
    q2 <- wtd_quantile(s2, w, probs = c(0.025, 0.975))

    v_PC1min <- PCAw$mu + q1[1] * PCAw$rotation[,1]
    v_PC1max <- PCAw$mu + q1[2] * PCAw$rotation[,1]
    v_PC2min <- PCAw$mu + q2[1] * PCAw$rotation[,2]
    v_PC2max <- PCAw$mu + q2[2] * PCAw$rotation[,2]

    shapes_mat <- rbind(
      PC1min = v_PC1min,
      PC1max = v_PC1max,
      PC2min = v_PC2min,
      PC2max = v_PC2max
    )

    shapes_arr <- geomorph::arrayspecs(shapes_mat, p, k)

    PC1min <- matrix(shapes_arr[,3,1], nrow = Y_LENGTH, ncol = X_LENGTH, byrow = TRUE)
    PC1max <- matrix(shapes_arr[,3,2], nrow = Y_LENGTH, ncol = X_LENGTH, byrow = TRUE)
    PC2min <- matrix(shapes_arr[,3,3], nrow = Y_LENGTH, ncol = X_LENGTH, byrow = TRUE)
    PC2max <- matrix(shapes_arr[,3,4], nrow = Y_LENGTH, ncol = X_LENGTH, byrow = TRUE)

    write.csv(PC1min, file.path(store.at, paste0("PC1min", CSV_SUFFIX, ".csv")), row.names = FALSE)
    write.csv(PC1max, file.path(store.at, paste0("PC1max", CSV_SUFFIX, ".csv")), row.names = FALSE)
    write.csv(PC2min, file.path(store.at, paste0("PC2min", CSV_SUFFIX, ".csv")), row.names = FALSE)
    write.csv(PC2max, file.path(store.at, paste0("PC2max", CSV_SUFFIX, ".csv")), row.names = FALSE)

    # resumen de pesos
    wsum <- data.frame(
      tip = tip,
      K_USE = length(wavs_k),
      TAU_MODE = TAU_MODE,
      tau = tau_use,
      n_eff = neff,
      d1 = d[1],
      dK = d[length(d)],
      d_median = median(d)
    )
    write.csv(wsum, file.path(store.at, paste0("WEIGHTS_summary", CSV_SUFFIX, ".csv")), row.names = FALSE)

    cat("saved weighted CSVs with suffix:", CSV_SUFFIX, "\n")
    flush.console()

  }, silent = TRUE)

  unlink(ali.tmp, recursive = TRUE)

  if (inherits(res, "try-error")) {
    cat("FAIL tip", tip, "\n")
    cat(as.character(res), "\n")
    flush.console()
  }
}


In [ ]:
# EXPORT: por-tip PNG (ΔPC1 + Mmean + ridge) + 2 colorbars

# PARÁMETROS
OUT_ROOT <- "../results/asr_KNN/SYNTH_TIPS"

MEAN_FILE <- "Mmean_wtd.csv"
OUT_NAME  <- "heat_dPC1_mean_wtd.png"

TLIM <- c(0, 5)
FLIM <- c(0, 10)
fmax_khz <- 8

DELTA_QCLIP <- 0.99
MEAN_QCLIP  <- 0.99
MEAN_ALPHA  <- 0.25
MEAN_GAMMA  <- 1.80

RIDGE_Q        <- 0.90
RIDGE_SMOOTH_K <- 9
RIDGE_USE_HALO <- FALSE
RIDGE_LWD      <- 3.2
RIDGE_HALO_LWD <- 5.8
RIDGE_HALO_COL <- "white"
RIDGE_COL      <- "black"

cmap_n    <- 201
COLS_DELTA <- colorRampPalette(c("blue","white","red"))(cmap_n)
COLS_MEAN_OVERLAY <- grDevices::adjustcolor(colorRampPalette(c("white","black"))(cmap_n), alpha.f = MEAN_ALPHA)
COLS_MEAN_BAR <- grDevices::adjustcolor(colorRampPalette(c("white","black"))(cmap_n),alpha.f = MEAN_ALPHA)


width  <- 2600
height <- 1800
res    <- 300

TICK_CEX <- 1.25
LAB_CEX  <- 1.35

BAR_TICK_CEX <- 1.10
BAR_LAB_CEX  <- 1.05

FIG_MAIN <- c(0.04, 0.99, 0.07, 0.99)

BAR_W_FRAC   <- 0.23
BAR_GAP_FRAC <- 0.035
BAR_Y0_FRAC  <- 0.10
BAR_Y1_FRAC  <- 0.60
BAR_MAR      <- c(1.10, 0.25, 0.55, 0.25)

# HELPERS
read_mat <- function(path) {
  m <- as.matrix(read.csv(path, check.names = FALSE))
  storage.mode(m) <- "numeric"
  m
}

ridge_from_mean <- function(mean_mat, y_vec, q = 0.90, smooth_k = 9) {
  nT <- ncol(mean_mat)
  r  <- rep(NA_real_, nT)

  for (j in 1:nT) {
    v <- mean_mat[, j]
    v <- v - min(v, na.rm = TRUE)
    v <- pmax(v, 0)
    s <- sum(v, na.rm = TRUE)
    if (!is.finite(s) || s <= 0) next
    cs <- cumsum(v) / s
    idx <- which(cs >= q)[1]
    if (!is.na(idx)) r[j] <- y_vec[idx]
  }

  # MOD: suavizado segmentado por bloques consecutivos
  if (smooth_k && smooth_k >= 3 && (smooth_k %% 2 == 1)) {
    ok <- which(is.finite(r))
    if (length(ok) >= smooth_k) {
      blocks <- split(ok, cumsum(c(TRUE, diff(ok) != 1)))
      for (b in blocks) {
        if (length(b) >= smooth_k) {
          r[b] <- stats::runmed(r[b], k = smooth_k, endrule = "median")
        }
      }
    }
  }
  r
}

plot_tip_panel <- function(d_full, m_full, out_png) {
  y_full <- seq(FLIM[1], FLIM[2], length.out = nrow(d_full))
  keep   <- which(y_full <= fmax_khz)

  d <- d_full[keep, , drop = FALSE]
  m <- m_full[keep, , drop = FALSE]

  x <- seq(TLIM[1], TLIM[2], length.out = ncol(d_full))
  y <- y_full[keep]

  # zlim ΔPC1 por tip
  M_tip <- as.numeric(stats::quantile(abs(d_full), probs = DELTA_QCLIP, na.rm = TRUE))
  if (!is.finite(M_tip) || M_tip == 0) M_tip <- max(abs(d_full), na.rm = TRUE)
  zlim_delta <- c(-M_tip, M_tip)

  # zlim Mean por tip
  vals_m <- as.numeric(m_full)
  vals_m <- vals_m[is.finite(vals_m)]
  if (length(vals_m) == 0) return(invisible(FALSE))

  lo_m <- as.numeric(stats::quantile(vals_m, probs = 0.10,       na.rm = TRUE))
  hi_m <- as.numeric(stats::quantile(vals_m, probs = MEAN_QCLIP, na.rm = TRUE))
  if (!is.finite(lo_m)) lo_m <- min(vals_m)
  if (!is.finite(hi_m)) hi_m <- max(vals_m)
  if (!is.finite(lo_m) || !is.finite(hi_m)) return(invisible(FALSE))
  if (hi_m <= lo_m) hi_m <- lo_m + 1e-6

  mm <- (m - lo_m) / (hi_m - lo_m)
  mm[!is.finite(mm)] <- 0
  mm[mm < 0] <- 0
  mm[mm > 1] <- 1
  mm <- mm^MEAN_GAMMA

  ridge_y <- ridge_from_mean(m, y, q = RIDGE_Q, smooth_k = RIDGE_SMOOTH_K)

  png(out_png, width = width, height = height, res = res)

  #  MAIN 
  par(fig = FIG_MAIN, new = FALSE)
  par(mar = c(6.8, 5.2, 1.6, 1.0) + 0.1, xaxs = "i", yaxs = "i")
  plot.new(); plot.window(xlim = range(x), ylim = c(FLIM[1], fmax_khz))

  # Base: Mmean (grises, con alpha)
  image(x, y, t(mm), col = COLS_MEAN_OVERLAY, zlim = c(0, 1),    axes = FALSE, xlab = "", ylab = "", add = TRUE)
  # Encima: ΔPC1 (rojo-azul)
  image(x, y, t(d),  col = COLS_DELTA,        zlim = zlim_delta, axes = FALSE, xlab = "", ylab = "", add = TRUE)

  # Ridge
  if (isTRUE(RIDGE_USE_HALO)) lines(x, ridge_y, col = RIDGE_HALO_COL, lwd = RIDGE_HALO_LWD)
  lines(x, ridge_y, col = RIDGE_COL, lwd = RIDGE_LWD)

  box()
  axis(1, at = 0:5, labels = 0:5, cex.axis = TICK_CEX)
  axis(2, at = seq(0, fmax_khz, by = 1), labels = seq(0, fmax_khz, by = 1), las = 1, cex.axis = TICK_CEX)
  mtext("Time (s)", side = 1, line = 2.9, cex = LAB_CEX, adj = 1)
  mtext("Frequency (kHz)", side = 2, line = 3.3, cex = LAB_CEX, adj = 1)

  #  BARRAS 
  plt <- par("plt")
  xL <- plt[1]; xR <- plt[2]
  fig_main <- par("fig")
  yB <- fig_main[3]
  yT <- plt[3]

  barY0 <- yB + BAR_Y0_FRAC * (yT - yB)
  barY1 <- yB + BAR_Y1_FRAC * (yT - yB)

  W <- (xR - xL)
  bw <- BAR_W_FRAC * W
  bg <- BAR_GAP_FRAC * W

  x0a <- xL
  x1a <- x0a + bw
  x0b <- x1a + bg
  x1b <- x0b + bw

  draw_bar <- function(fig, cols, labels_at, labels_txt, title) {
    par(fig = fig, new = TRUE)
    par(mar = BAR_MAR, xaxs = "i", yaxs = "i")
    par(xpd = NA)

    plot.new(); plot.window(xlim = TLIM, ylim = c(0, 1))

    xb <- seq(TLIM[1], TLIM[2], length.out = cmap_n)
    image(x = xb, y = 0.5,
          z = matrix(xb, nrow = length(xb), ncol = 1),
          col = cols, add = TRUE)

    axis(1, at = labels_at, labels = labels_txt, cex.axis = BAR_TICK_CEX)
    mtext(title, side = 3, line = 0.05, cex = BAR_LAB_CEX)
  }

  draw_bar(fig = c(x0a, x1a, barY0, barY1),
           cols = COLS_DELTA,
           labels_at = c(TLIM[1], mean(TLIM), TLIM[2]),
           labels_txt = c("min", "0", "max"),
           title = "ΔPC1")

  draw_bar(fig = c(x0b, x1b, barY0, barY1),
           cols = COLS_MEAN_BAR,
           labels_at = c(TLIM[1], TLIM[2]),
           labels_txt = c("min", "max"),
           title = "Mean")

  dev.off()
  invisible(TRUE)
}

# LOOP TIPS
tip.dirs <- list.dirs(OUT_ROOT, recursive = FALSE, full.names = TRUE)
tip.dirs <- tip.dirs[grepl("^tip_", basename(tip.dirs))]
tip.ids  <- sort(gsub("^tip_", "", basename(tip.dirs)))

ok_n <- 0
for (tip_id in tip.ids) {
  store.at <- file.path(OUT_ROOT, paste0("tip_", tip_id), "store.at")
  f_min  <- file.path(store.at, "PC1min_wtd.csv")
  f_max  <- file.path(store.at, "PC1max_wtd.csv")
  f_mean <- file.path(store.at, MEAN_FILE)

  if (!file.exists(f_min) || !file.exists(f_max) || !file.exists(f_mean)) next

  d_full <- read_mat(f_max) - read_mat(f_min)
  m_full <- read_mat(f_mean)
  if (!all(dim(d_full) == dim(m_full))) next

  out_png <- file.path(store.at, OUT_NAME)
  if (isTRUE(plot_tip_panel(d_full, m_full, out_png))) {
    ok_n <- ok_n + 1
    cat("saved:", out_png, "\n")
  }
}

cat("DONE. Panels saved:", ok_n, "\n")


In [ ]:
# EXPORT “FIGURA PURA” por-tip (sin ejes/ticks/título/barras)

# PARÁMETROS
OUT_ROOT <- "../results/asr_KNN/SYNTH_TIPS"

MEAN_FILE <- "Mmean_wtd.csv"
OUT_NAME  <- "heat_dPC1_mean_wtd__PURE.png"

TLIM <- c(0, 5)
FLIM <- c(0, 10)
fmax_khz <- 8

DELTA_QCLIP <- 0.99
MEAN_QCLIP  <- 0.99
MEAN_ALPHA  <- 0.25
MEAN_GAMMA  <- 1.80

RIDGE_Q        <- 0.90
RIDGE_SMOOTH_K <- 9
RIDGE_USE_HALO <- FALSE
RIDGE_LWD      <- 3.2
RIDGE_HALO_LWD <- 5.8
RIDGE_HALO_COL <- "white"
RIDGE_COL      <- "black"

cmap_n    <- 201
COLS_DELTA <- colorRampPalette(c("blue","white","red"))(cmap_n)
COLS_MEAN_OVERLAY <- grDevices::adjustcolor(colorRampPalette(c("white","black"))(cmap_n), alpha.f = MEAN_ALPHA)

width  <- 2600
height <- 1800
res    <- 300

OUT_PURE_DIR <- file.path(OUT_ROOT, "_FIGURAS_LIMPIAS")
dir.create(OUT_PURE_DIR, recursive = TRUE, showWarnings = FALSE)

# HELPERS
read_mat <- function(path) {
  m <- as.matrix(read.csv(path, check.names = FALSE))
  storage.mode(m) <- "numeric"
  m
}

ridge_from_mean <- function(mean_mat, y_vec, q = 0.90, smooth_k = 9) {
  nT <- ncol(mean_mat)
  r  <- rep(NA_real_, nT)

  for (j in 1:nT) {
    v <- mean_mat[, j]
    v <- v - min(v, na.rm = TRUE)
    v <- pmax(v, 0)
    s <- sum(v, na.rm = TRUE)
    if (!is.finite(s) || s <= 0) next
    cs <- cumsum(v) / s
    idx <- which(cs >= q)[1]
    if (!is.na(idx)) r[j] <- y_vec[idx]
  }

  # MOD: suavizado segmentado por bloques consecutivos
  if (smooth_k && smooth_k >= 3 && (smooth_k %% 2 == 1)) {
    ok <- which(is.finite(r))
    if (length(ok) >= smooth_k) {
      blocks <- split(ok, cumsum(c(TRUE, diff(ok) != 1)))
      for (b in blocks) {
        if (length(b) >= smooth_k) {
          r[b] <- stats::runmed(r[b], k = smooth_k, endrule = "median")
        }
      }
    }
  }
  r
}

plot_tip_pure <- function(d_full, m_full, out_png) {
  y_full <- seq(FLIM[1], FLIM[2], length.out = nrow(d_full))
  keep   <- which(y_full <= fmax_khz)

  d <- d_full[keep, , drop = FALSE]
  m <- m_full[keep, , drop = FALSE]

  x <- seq(TLIM[1], TLIM[2], length.out = ncol(d_full))
  y <- y_full[keep]

  # zlim ΔPC1 por tip
  M_tip <- as.numeric(stats::quantile(abs(d_full), probs = DELTA_QCLIP, na.rm = TRUE))
  if (!is.finite(M_tip) || M_tip == 0) M_tip <- max(abs(d_full), na.rm = TRUE)
  zlim_delta <- c(-M_tip, M_tip)

  # normalización Mean por tip
  vals_m <- as.numeric(m_full)
  vals_m <- vals_m[is.finite(vals_m)]
  if (length(vals_m) == 0) return(invisible(FALSE))

  lo_m <- as.numeric(stats::quantile(vals_m, probs = 0.10,       na.rm = TRUE))
  hi_m <- as.numeric(stats::quantile(vals_m, probs = MEAN_QCLIP, na.rm = TRUE))
  if (!is.finite(lo_m)) lo_m <- min(vals_m)
  if (!is.finite(hi_m)) hi_m <- max(vals_m)
  if (!is.finite(lo_m) || !is.finite(hi_m)) return(invisible(FALSE))
  if (hi_m <= lo_m) hi_m <- lo_m + 1e-6

  mm <- (m - lo_m) / (hi_m - lo_m)
  mm[!is.finite(mm)] <- 0
  mm[mm < 0] <- 0
  mm[mm > 1] <- 1
  mm <- mm^MEAN_GAMMA

  ridge_y <- ridge_from_mean(m, y, q = RIDGE_Q, smooth_k = RIDGE_SMOOTH_K)

  png(out_png, width = width, height = height, res = res)

  par(mar = c(0, 0, 0, 0), xaxs = "i", yaxs = "i")
  plot.new()
  plot.window(xlim = range(x), ylim = c(FLIM[1], fmax_khz))

  image(x, y, t(mm), col = COLS_MEAN_OVERLAY, zlim = c(0, 1),
        axes = FALSE, xlab = "", ylab = "", add = TRUE)

  image(x, y, t(d),  col = COLS_DELTA, zlim = zlim_delta,
        axes = FALSE, xlab = "", ylab = "", add = TRUE)

  if (isTRUE(RIDGE_USE_HALO)) lines(x, ridge_y, col = RIDGE_HALO_COL, lwd = RIDGE_HALO_LWD)
  lines(x, ridge_y, col = RIDGE_COL, lwd = RIDGE_LWD)

  dev.off()
  invisible(TRUE)
}

# LOOP TIPS
tip.dirs <- list.dirs(OUT_ROOT, recursive = FALSE, full.names = TRUE)
tip.dirs <- tip.dirs[grepl("^tip_", basename(tip.dirs))]
tip.ids  <- sort(gsub("^tip_", "", basename(tip.dirs)))

ok_n <- 0
for (tip_id in tip.ids) {
  store.at <- file.path(OUT_ROOT, paste0("tip_", tip_id), "store.at")
  f_min  <- file.path(store.at, "PC1min_wtd.csv")
  f_max  <- file.path(store.at, "PC1max_wtd.csv")
  f_mean <- file.path(store.at, MEAN_FILE)

  if (!file.exists(f_min) || !file.exists(f_max) || !file.exists(f_mean)) next

  d_full <- read_mat(f_max) - read_mat(f_min)
  m_full <- read_mat(f_mean)
  if (!all(dim(d_full) == dim(m_full))) next

  out_png <- file.path(OUT_PURE_DIR, paste0("tip_", tip_id, "__", OUT_NAME))
  if (isTRUE(plot_tip_pure(d_full, m_full, out_png))) {
    ok_n <- ok_n + 1
    cat("saved:", out_png, "\n")
  }
}

cat("DONE. Pure panels saved:", ok_n, "\n")
cat("Folder:", OUT_PURE_DIR, "\n")
